## Coin Tosses
A jar has 1000 coins, of which 999 are fair and 1 is double-headed. Pick a coin at random, and toss it 10 times. Given that you see 10 heads, what is the probability that the next toss of that coin is also a head? Give your answer to 3 significant figures.

Let $D$ be the event that the double-headed coin was picked, and let $F$ be the event that the fair coin was picked

$p(D) = 1/1000$
$p(F) = 999/1000$

Apply Bayes Theorem to calculate the posterior probabilities of each event,

$$P(D|data) = \frac{p(Data|D)\times P(D)}{p(Data|D)\times P(D) + p(Data|F)\times P(F)}$$

$$P(F|data) = \frac{p(Data|F)\times P(F)}{p(Data|F)\times P(F) + p(Data|D)\times P(D)}$$

The previous two formulas show that $P(F|data) = 1-P(D|data)$

Here the data is ten coin tosses resulting in 10 heads.     
Therefore, 
$p(Data|D) = 1^{10} = 1 $  and  $ p(Data|F) = (0.5)^{10} \approx 0.00098$.

Thus, we have all the ingredients to calculate the two posteriors. Next, we need to calculate the probability of getting a head on the 11th toss, which we will do using the law of total probability.

$$p(H) = p(H|D)p(D|data) + p(H|F)(F|data)$$


In [1]:
class BayesCoinToss:
    """Obtains posterior probability of a coin."""
    
    def __init__(self, prior1, prob_head1, prob_head2, n_heads, n_tails):
        """ prior1, prob_head1, and prob_head2 must be in [0, 1].
        n_heads and n_tails must be non-negative"""
        if prior1 < 0 or prior1 > 1:
            raise ValueError('The prior must be in [0, 1].')
        if (prob_head1 < 0 or prob_head1 > 1) or \
           (prob_head2 < 0 or prob_head2 > 1):
            raise ValueError('prob(head) must be in [0, 1].')
        if type(n_heads) != int or type(n_tails) != int:
            raise TypeError('Number of heads or tails must be of type int.')
        if n_heads < 0 or n_tails < 0:
            raise ValueError('Number of heads or tails must be positive')
        self.prior1 = prior1
        self.prior2 = 1 - prior1
        self.prob_head1 = prob_head1
        self.prob_head2 = prob_head2
        self.n_heads = n_heads
        self.n_tails = n_tails
        
    def get_likelihood(self):
        self.likelihood1 = self.prob_head1**self.n_heads * \
                           (1 - self.prob_head1)**self.n_tails
        self.likelihood2 = self.prob_head2**self.n_heads * \
                           (1 - self.prob_head2)**self.n_tails
        return self.likelihood1, self.likelihood2
    
    def get_posterior(self):
        if hasattr(self, 'likelihood1'): 
            term1 = self.likelihood1 * self.prior1
            term2 = self.likelihood2 * self.prior2
            self.posterior1 = term1 / (term1 + term2)
            return self.posterior1
        else:
            raise AttributeError('please calculate the likelihoods first!')
    
    def get_prob_head_next_toss(self):
        if hasattr(self, 'poststerior1'):
            prob_head_next_toss = self.prob_head1 * self.posterior1 + \
                                  self.prob_head2 * (1 - self.posterior1)
            return prob_head_next_toss
        else:
            raise AttributeError('Please calculate the posterior first!')


In [2]:
PRIOR1 = .001
PROB_HEAD1 = 1
PROB_HEAD2 = .5
N_HEADS = 10
N_TAILS = 0

# BayesCoinToss(prior1, prob_head1, prob_head2, n_heads, n_tails)
bayes = BayesCoinToss(PRIOR1, PROB_HEAD1, PROB_HEAD2, 
                      N_HEADS, N_TAILS)
L1, L2 = bayes.get_likelihood()
posterior = bayes.get_posterior()

print('p(D), p(H|D), p(data|D):', 
      (bayes.prior1, bayes.prob_head1, bayes.likelihood1))
print('p(F), p(H|F), p(data|F):',
      (bayes.prior2, bayes.prob_head2, bayes.likelihood2))
print('p(D|data) =', posterior)
prob_head_given_data = PROB_HEAD1 * posterior + PROB_HEAD2 * (1 - posterior)
print('Probability of a head on next toss =', prob_head_given_data)

p(D), p(H|D), p(data|D): (0.001, 1, 1)
p(F), p(H|F), p(data|F): (0.999, 0.5, 0.0009765625)
p(D|data) = 0.5061789421651013
Probability of a head on next toss = 0.7530894710825506


In [3]:
# test the function under no observations assumption
PRIOR1 = .001
PROB_HEAD1 = 1
PROB_HEAD2 = .5
N_HEADS = 0
N_TAILS = 0

# BayesCoinToss(prior1, prob_head1, prob_head2, n_heads, n_tails)
bayes = BayesCoinToss(PRIOR1, PROB_HEAD1, PROB_HEAD2, 
                      N_HEADS, N_TAILS)
L1, L2 = bayes.get_likelihood()
posterior = bayes.get_posterior()
print('When no data have been observed, the posterior is the prior!')
print('p(D), p(H|D), p(data|D):', 
      (bayes.prior1, bayes.prob_head1, bayes.likelihood1))
print('p(F), p(H|F), p(data|F):',
      (bayes.prior2, bayes.prob_head2, bayes.likelihood2))
print('p(D|data) =', posterior)


When no data have been observed, the posterior is the prior!
p(D), p(H|D), p(data|D): (0.001, 1, 1)
p(F), p(H|F), p(data|F): (0.999, 0.5, 1.0)
p(D|data) = 0.001


In [4]:
# test the function under equal priors and two fair coins
PRIOR1 = .5
PROB_HEAD1 = .5
PROB_HEAD2 = .5
N_HEADS = 1
N_TAILS = 0

# BayesCoinToss(prior1, prob_head1, prob_head2, n_heads, n_tails)
bayes = BayesCoinToss(PRIOR1, PROB_HEAD1, PROB_HEAD2, 
                      N_HEADS, N_TAILS)
L1, L2 = bayes.get_likelihood()
posterior = bayes.get_posterior()
print('When both coins are fair and the priors are equal,\nthe posterior is 0.5')
print('p(D), p(H|D), p(data|D):', 
      (bayes.prior1, bayes.prob_head1, bayes.likelihood1))
print('p(F), p(H|F), p(data|F):',
      (bayes.prior2, bayes.prob_head2, bayes.likelihood2))
print('p(D|data) =', posterior)


When both coins are fair and the priors are equal,
the posterior is 0.5
p(D), p(H|D), p(data|D): (0.5, 0.5, 0.5)
p(F), p(H|F), p(data|F): (0.5, 0.5, 0.5)
p(D|data) = 0.5


## Should I bring an Umbrella to Seattle?

You're about to get on a plane to Seattle. You want to know  if you should bring an umbrella. You call 3 random friends of yours who live there and ask each independently if it's raining. Each of your friends has a 2/3 chance of telling you the truth and a 1/3 chance of messing with you by lying. All 3 friends tell you that "Yes" it is raining. What is the probability that it's actually raining in Seattle?

$p(yes|~\text{rain}) = 2/3$
$p(yes|~\text{no rain}) = 1/3$

$$p(\text{rain}|~ yes, yes, yes)  = \frac{p(yes, yes, yes |~ \text{rain}) \times p(\text{rain})}{p(yes, yes, yes)}\\
= \frac{p(yes, yes, yes |~ rain) \times p(\text{rain})}{p(yes, yes, yes |~ rain) p(\text{rain}) + p(yes, yes, yes |~ \text{no rain}) p(\text{no rain})}$$
 
$$= \frac{(2/3) (2/3) (2/3)\times p(\text{rain})}{(2/3) (2/3) (2/3)\times p(\text{rain}) + (1/3) (1/3) (1/3)\times p(\text{no rain})} \\
=\frac{(8/27)\times p(\text{rain})}{(8/27)\times p(\text{rain}) + (1/27)\times p(\text{no rain})}$$

Suppose that rain/no rain is a 50-50 chance type of a scenario. Thus $p(\text{rain}) = p(\text{no rain}) = 0.5$. Thus,

$$p(\text{rain}|~ yes, yes, yes) = \frac{(8/27)(1/2)}{(8/27)(1/2) + (1/27)(1/2)} = \frac{(8/27)}{(9/27)} = \frac{8}{9} = 0.888\dots$$

So given your friends' responses, you should bring an umbrella -- although it is not a bad idea to always bring one to Seattle.

In [5]:
# notice we can use BayesCoinToss since we have 
# a binary outcome (rain/no rain) 

PRIOR1 = 1/2
PROB_HEAD1 = 2/3
PROB_HEAD2 = 1/3
N_HEADS = 3
N_TAILS = 0

bayes = BayesCoinToss(PRIOR1, PROB_HEAD1, PROB_HEAD2, 
                      N_HEADS, N_TAILS)
L1, L2 = bayes.get_likelihood()
posterior = bayes.get_posterior()
print("probability of rain given", N_HEADS, "yes's =", posterior)

probability of rain given 3 yes's = 0.8888888888888888
